# Communication with SAP HANA Databases

In [ ]:
# Download required packages

%pip install sqlalchemy-hana pandas python-dotenv

## Connect to DB

There are two different styles of connections:
1. dbapi
    - Low-level connection using the hdbcli driver
    - Used to send simple SQL commands
    - Requires manual closing
2. SQLAlchemy Engine (Preferred Method)
    - High-level connection built on top of the hdbcli driver
    - The engine manages a connection pool
    - Automatic closing 

In [ ]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv(override=True)  # reads variables from a .env file and sets them in os.environ

engine = create_engine(f'hana://{os.getenv("HANA_USER")}:{os.getenv("HANA_PASSWORD")}@{os.getenv("HANA_ADDRESS")}:{os.getenv("HANA_PORT")}')

## Read from DB

In [ ]:
import pandas as pd

sql_command = 'SELECT * FROM "_SYS_BIC"."system-local.bw.bw2hana/ZADSO_TTC"'
df = pd.read_sql(sql_command, engine)

## Analyze/Edit the Data

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop(columns=['1ROWCOUNT'], inplace=True)

In [ ]:
df.columns

In [ ]:
df['SURNAME'] = df['4ZADSO_TTC_NAME'].str.split(',').str[0]

In [ ]:
df.columns

## Write to DB

In [ ]:
df.to_sql(name="processed_ttc_data", 
                con=engine, 
                schema=os.getenv("HANA_USER"), 
                if_exists='replace', 
                index=False)